In [1]:
import numpy as np

class Boids:
    def __init__(self, num_boids=128, width=500, height=500):
        self.num_boids = num_boids
        self.width = width
        self.height = height
        self.positions = np.random.rand(num_boids, 2) * np.array([width, height])
        self.velocities = np.random.rand(num_boids, 2) * 2 - 1

    def update(self):
        # Simple flocking rules: cohesion, alignment, and separation
        cohesion_strength = 0.01
        alignment_strength = 0.05
        separation_strength = 0.1

        # Cohesion: Move towards the center of mass
        center_of_mass = np.mean(self.positions, axis=0)
        cohesion = (center_of_mass - self.positions) * cohesion_strength

        # Alignment: Align with the average velocity of neighbors
        average_velocity = np.mean(self.velocities, axis=0)
        alignment = (average_velocity - self.velocities) * alignment_strength

        # Separation: Avoid crowding neighbors
        separation = np.zeros_like(self.positions)
        for i in range(self.num_boids):
            distances = np.linalg.norm(self.positions[i] - self.positions, axis=1)
            too_close = distances < 25
            too_close[i] = False  # Ignore self
            if np.any(too_close):
                separation[i] = -np.mean(self.positions[too_close] - self.positions[i], axis=0) * separation_strength

        # Update velocities and positions
        self.velocities += cohesion + alignment + separation
        self.positions += self.velocities

        # Wrap around the edges of the screen
        self.positions %= np.array([self.width, self.height])

    def render(self):
        # Render the boids as points on a 2D plane
        import matplotlib.pyplot as plt
        plt.scatter(self.positions[:, 0], self.positions[:, 1], s=10)
        plt.xlim(0, self.width)
        plt.ylim(0, self.height)
        plt.show()


In [13]:
import torch
import open_clip
from PIL import Image
import matplotlib.pyplot as plt

# Load the CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
model.eval()  # model in train mode by default, impacts some models with BatchNorm or stochastic depth active
tokenizer = open_clip.get_tokenizer('ViT-B-32')

# Define a function to evaluate the simulation
def evaluate_simulation(boids, target_prompt):
    # Render the simulation as an image
    plt.scatter(boids.positions[:, 0], boids.positions[:, 1], s=10)
    plt.xlim(0, boids.width)
    plt.ylim(0, boids.height)
    plt.savefig("boids.png")
    plt.close()

    # Preprocess the image and prompt
    image = preprocess(Image.open("boids.png")).unsqueeze(0).to(device)
    print(type(image.data))
    text = tokenizer([target_prompt]).to(device)
    print(type(text.data))

    # Compute the similarity score
    with torch.no_grad(), torch.cuda.amp.autocast():
        image_features = model.encode_image(image).to(device)
        print(type(image_features.data))
        text_features = model.encode_text(text).to(device)
        print(type(text_features.data))
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)

        similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)

    return similarity

# Run the simulation and evaluate it
boids = Boids()
for _ in range(100):  # Run for 100 steps
    boids.update()

similarity = evaluate_simulation(boids, "a flock of birds")
print(f"Similarity to 'a flock of birds': {similarity}")


<class 'torch.Tensor'>
<class 'torch.Tensor'>


RuntimeError: Input type (torch.cuda.HalfTensor) and weight type (torch.FloatTensor) should be the same

In [2]:
boids = Boids()
for _ in range(100):  # Run for 100 steps
    boids.update()

similarity = evaluate_simulation(boids, "a flock of birds")
print(f"Similarity to 'a flock of birds': {similarity}")


NameError: name 'evaluate_simulation' is not defined